In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold  #交叉验证
from sklearn.metrics import confusion_matrix

import pandas as pd
import numpy as np
import graphviz
import pickle
import warnings

warnings.filterwarnings("ignore")

In [2]:
disorder_R_H = pickle.load(open('/home/dldx/R-H/code/classification/ML/data/disorder/avg_R-H_all_dsiorder.pkl','rb'))

data1 = pd.read_csv('/home/dldx/R-H/code/classification/ML/data/disorder/MSA_disorder_R_H.csv')
input_lable = np.array(data1.Tier)

In [3]:
all_21_IDR_disorder = []
for position in range(21):
    one_IDR = []
    R_IDR = [i[position] for i in disorder_R_H[0]]
    H_IDR = [i[position] for i in disorder_R_H[1]]
 
    # 转化为二维数组
    for i in range(72):
        test =[]
        test.append(R_IDR[i])
        one_IDR.append(test)
    for i in range(107):
        test =[]
        test.append(H_IDR[i])
        one_IDR.append(test)

    # 所有IDR的平均disorder值添加到一个数组
    all_21_IDR_disorder.append(one_IDR)

In [4]:
disorder_R_H[0][65][15]

0.03288253745995458

In [6]:
# 重新跑关于IDR16由数据更改的情况

# 评估结果
all_21_IDR_length_evaluate = []

# 设置超参数  n_estimators：决策数的数量 默认100
param_dict = {"n_estimators": [100, 200, 400, 700], "max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 5)}

# 21个IDR区域分别进行分类
one_evaluate = []
for r in range(0,31):
    one_random_state = []
    # train_test_split 按比例划分原数据集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(all_21_IDR_disorder[15],input_lable,train_size=0.8,random_state = r)
    # 定义模型
    model = RandomForestClassifier(criterion="entropy")
    # 将模型加入超参数网格搜索和交叉验证     
    model = GridSearchCV(model, param_grid=param_dict, cv=5)
    # 训练数据集
    model = model.fit(Xtrain, Ytrain)
    # 最优的模型参数
    params = model.best_params_ 
    # 模型评估
    predict = model.predict(Xtest)
    cm = confusion_matrix(Ytest,predict)
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
    one_random_state.append(accuracy)
    one_random_state.append(mcc)
    one_random_state.append(sens)
    one_random_state.append(spec)
    one_evaluate.append(one_random_state)

    print("数据集划分 random_state：" + str(r) + " 时，")
    print("Evaluate：")
    print("最优的模型参数是：" + str(params))
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")

all_21_IDR_length_evaluate.append(one_evaluate)       
print("以上为IDR__" + str(15 + 1) + "时，evaluation")

数据集划分 random_state：0 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'n_estimators': 100, 'random_state': 25}
  accuracy : 0.7222222222222222
  mcc : 0.5459486832355505
  sens : 46.666666666666664
  spec : 90.47619047619048
数据集划分 random_state：1 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'n_estimators': 100, 'random_state': 25}
  accuracy : 0.6111111111111112
  mcc : 0.27112768548339117
  sens : 41.17647058823529
  spec : 78.94736842105263
数据集划分 random_state：2 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'n_estimators': 200, 'random_state': 15}
  accuracy : 0.6111111111111112
  mcc : 0.3572172541558801
  sens : 45.0
  spec : 81.25
数据集划分 random_state：3 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'n_estimators': 100, 'random_state': 5}
  accuracy : 0.6666666666666666
  mcc : 0.325
  sens : 62.5
  spec : 70.0
数据集划分 random_state：4 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'n_estimators': 100, 'random_state': 15}
  accuracy : 0.6666666666666666
  mcc : 0.19245008972987526
  sens : 36.36363636363637
  spec : 80.0
数据集划分

## 重跑数据在RF_replenish

In [5]:
# 评估结果
all_21_IDR_length_evaluate = []

# 设置超参数  n_estimators：决策数的数量 默认100
param_dict = {"n_estimators": [100, 200, 400, 700], "max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 5)}

# 21个IDR区域分别进行分类
for position in range(21):
    one_evaluate = []
    for r in range(0,31):
        one_random_state = []
        # train_test_split 按比例划分原数据集
        Xtrain,Xtest,Ytrain,Ytest = train_test_split(all_21_IDR[position],input_lable,train_size=0.8,random_state = r)
        # 定义模型
        model = RandomForestClassifier(criterion="entropy")
        # 将模型加入超参数网格搜索和交叉验证     
        model = GridSearchCV(model, param_grid=param_dict, cv=5)
        # 训练数据集
        model = model.fit(Xtrain, Ytrain)
        # 最优的模型参数
        params = model.best_params_ 
        # 模型评估
        predict = model.predict(Xtest)
        cm = confusion_matrix(Ytest,predict)
        tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
        n = tp + fp + tn + fn
        accuracy = (tp + tn)/n 
        mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
        sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
        spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
        one_random_state.append(accuracy)
        one_random_state.append(mcc)
        one_random_state.append(sens)
        one_random_state.append(spec)
        one_evaluate.append(one_random_state)
        
        print("数据集划分 random_state：" + str(r) + " 时，")
        print("Evaluate：")
        print("最优的模型参数是：" + str(params))
        print("  accuracy : " + str(accuracy))
        print("  mcc : " + str(mcc))
        print("  sens : " + str(sens))
        print("  spec : " + str(spec)) 
        print("============================")
        
    all_21_IDR_length_evaluate.append(one_evaluate)       
    print("以上为IDR__" + str(position + 1) + "时，evaluation")
    print("***********************************************************************")

数据集划分 random_state：0 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.4341647670391649
  sens : 33.33333333333333
  spec : 90.47619047619048
数据集划分 random_state：1 时，
Evaluate：
  accuracy : 0.4444444444444444
  mcc : -0.22075539284417395
  sens : 17.647058823529413
  spec : 68.42105263157895
数据集划分 random_state：2 时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : -0.03207914062416534
  sens : 35.0
  spec : 62.5
数据集划分 random_state：3 时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : 0.0816496580927726
  sens : 18.75
  spec : 85.0
数据集划分 random_state：4 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : -0.00784464540552736
  sens : 27.27272727272727
  spec : 72.0
数据集划分 random_state：5 时，
Evaluate：
  accuracy : 0.4166666666666667
  mcc : -0.30996520993903337
  sens : 10.526315789473683
  spec : 76.47058823529412
数据集划分 random_state：15 时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : -0.11572751247156893
  sens : 16.666666666666664
  spec : 75.0
数据集划分 random_state：16 时，
Evaluate：

数据集划分 random_state：26 时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : 0.4472135954999579
  sens : 33.33333333333333
  spec : 91.66666666666666
数据集划分 random_state：27 时，
Evaluate：
  accuracy : 0.75
  mcc : 0.29095718698132317
  sens : 50.0
  spec : 82.14285714285714
数据集划分 random_state：28 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.19702760155977517
  sens : 30.76923076923077
  spec : 82.6086956521739
数据集划分 random_state：29 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.24033535298268816
  sens : 42.857142857142854
  spec : 77.27272727272727
数据集划分 random_state：30 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : -0.08206099398622183
  sens : 21.428571428571427
  spec : 72.72727272727273
以上为IDR__6时，evaluation
***********************************************************************
数据集划分 random_state：0 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : 0.2586743945509748
  sens : 20.0
  spec : 90.47619047619048
数据集划分 random_state：1 时，
Evaluate：
  accuracy : 0.6111

数据集划分 random_state：12 时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : -0.11697953037312037
  sens : 23.52941176470588
  spec : 68.42105263157895
数据集划分 random_state：13 时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : 0.04351941398892446
  sens : 35.714285714285715
  spec : 68.18181818181817
数据集划分 random_state：14 时，
Evaluate：
  accuracy : 0.4444444444444444
  mcc : -0.1785061808835215
  sens : 26.666666666666668
  spec : 57.14285714285714
数据集划分 random_state：15 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.22360679774997896
  sens : 25.0
  spec : 87.5
数据集划分 random_state：16 时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : -0.21721170701921008
  sens : 7.6923076923076925
  spec : 82.6086956521739
数据集划分 random_state：17 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : 0.04403865954435934
  sens : 40.0
  spec : 65.38461538461539
数据集划分 random_state：18 时，
Evaluate：
  accuracy : 0.3611111111111111
  mcc : -0.4652421051992354
  sens : 4.545454545454546
  spec : 85.7142857

数据集划分 random_state：30 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.23937535164005458
  sens : 50.0
  spec : 72.72727272727273
以上为IDR__9时，evaluation
***********************************************************************
数据集划分 random_state：0 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.2592063157538598
  sens : 53.333333333333336
  spec : 71.42857142857143
数据集划分 random_state：1 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.36893544194599504
  sens : 41.17647058823529
  spec : 84.21052631578947
数据集划分 random_state：2 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : 0.25663312499332275
  sens : 45.0
  spec : 75.0
数据集划分 random_state：3 时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : 0.09128709291752769
  sens : 37.5
  spec : 70.0
数据集划分 random_state：4 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.13335897189396514
  sens : 36.36363636363637
  spec : 76.0
数据集划分 random_state：5 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : 0.12366938848016847
 

数据集划分 random_state：16 时，
Evaluate：
  accuracy : 0.75
  mcc : 0.44232586846469135
  sens : 84.61538461538461
  spec : 69.56521739130434
数据集划分 random_state：17 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : 0.18808016817268094
  sens : 80.0
  spec : 50.0
数据集划分 random_state：18 时，
Evaluate：
  accuracy : 0.4444444444444444
  mcc : 0.025253813613805267
  sens : 22.727272727272727
  spec : 78.57142857142857
数据集划分 random_state：19 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.28210976821983985
  sens : 61.53846153846154
  spec : 69.56521739130434
数据集划分 random_state：20 时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : 0.47925723781702034
  sens : 83.33333333333334
  spec : 75.0
数据集划分 random_state：21 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : 0.1851640199545103
  sens : 50.0
  spec : 66.66666666666666
数据集划分 random_state：22 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.23417772655640703
  sens : 54.54545454545454
  spec : 72.0
数据集划分 random_state：23 时，
Evaluate：


...

数据集划分 random_state：19 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : 0.1203858530857692
  sens : 30.76923076923077
  spec : 78.26086956521739
数据集划分 random_state：20 时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : 0.3849001794597505
  sens : 83.33333333333334
  spec : 66.66666666666666
数据集划分 random_state：21 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : 0.09072184232530289
  sens : 27.77777777777778
  spec : 77.77777777777779
数据集划分 random_state：22 时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : 0.35787733998702753
  sens : 72.72727272727273
  spec : 72.0
数据集划分 random_state：23 时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : 0.5237849266164972
  sens : 42.857142857142854
  spec : 90.9090909090909
数据集划分 random_state：24 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : -0.03876376661011099
  sens : 28.57142857142857
  spec : 68.18181818181817
数据集划分 random_state：25 时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.4103049699311091
  sens : 42.857142857142854

数据集划分 random_state：5 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : 0.2475368857441686
  sens : 42.10526315789473
  spec : 76.47058823529412
数据集划分 random_state：6 时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.3624826753406541
  sens : 57.14285714285714
  spec : 77.27272727272727
数据集划分 random_state：7 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : 0.15496384542845748
  sens : 87.5
  spec : 42.857142857142854
数据集划分 random_state：8 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.3177719638026945
  sens : 42.857142857142854
  spec : 81.81818181818183
数据集划分 random_state：9 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : 0.2142220429979365
  sens : 47.368421052631575
  spec : 70.58823529411765
数据集划分 random_state：10 时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : 0.1
  sens : 50.0
  spec : 60.0
数据集划分 random_state：11 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.4330127018922193
  sens : 50.0
  spec : 83.33333333333334
数据集划分 random_state：12 时，
Evaluat

数据集划分 random_state：23 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : -0.03876376661011099
  sens : 28.57142857142857
  spec : 68.18181818181817
数据集划分 random_state：24 时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.4899559349388659
  sens : 21.428571428571427
  spec : 95.45454545454545
数据集划分 random_state：25 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : 0.1556997888323046
  sens : 21.428571428571427
  spec : 86.36363636363636
数据集划分 random_state：26 时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : 0.0
  sens : 33.33333333333333
  spec : 66.66666666666666
数据集划分 random_state：27 时，
Evaluate：
  accuracy : 0.5
  mcc : -0.11511756628934004
  sens : 25.0
  spec : 57.14285714285714
数据集划分 random_state：28 时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : -0.3349469226516178
  sens : 7.6923076923076925
  spec : 69.56521739130434
数据集划分 random_state：29 时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : 0.4682955256039708
  sens : 50.0
  spec : 86.36363636363636
数据集划分 random

数据集划分 random_state：9 时，
Evaluate：
  accuracy : 0.4444444444444444
  mcc : -0.140028008402801
  sens : 21.052631578947366
  spec : 70.58823529411765
数据集划分 random_state：10 时，
Evaluate：
  accuracy : 0.3888888888888889
  mcc : -0.21081851067789195
  sens : 37.5
  spec : 40.0
数据集划分 random_state：11 时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : 0.375
  sens : 16.666666666666664
  spec : 94.44444444444444
数据集划分 random_state：12 时，
Evaluate：
  accuracy : 0.4444444444444444
  mcc : -0.4038019652808613
  sens : 5.88235294117647
  spec : 78.94736842105263
数据集划分 random_state：13 时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : 0.11626802793945508
  sens : 42.857142857142854
  spec : 68.18181818181817
数据集划分 random_state：14 时，
Evaluate：
  accuracy : 0.3888888888888889
  mcc : -0.6078306738548308
  sens : 0.0
  spec : 66.66666666666666
数据集划分 random_state：15 时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : 0.125
  sens : 41.66666666666667
  spec : 70.83333333333334
数据集划分 random_state：16 时，

数据集划分 random_state：26 时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : -0.048038446141526144
  sens : 25.0
  spec : 70.83333333333334
数据集划分 random_state：27 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : -0.1002660580490852
  sens : 12.5
  spec : 78.57142857142857
数据集划分 random_state：28 时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.19702760155977517
  sens : 30.76923076923077
  spec : 82.6086956521739
数据集划分 random_state：29 时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.3798849127790877
  sens : 50.0
  spec : 81.81818181818183
数据集划分 random_state：30 时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : -0.5195887333176439
  sens : 0.0
  spec : 86.36363636363636
以上为IDR__20时，evaluation
***********************************************************************
数据集划分 random_state：0 时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : -0.051434449987363975
  sens : 46.666666666666664
  spec : 47.61904761904761
数据集划分 random_state：1 时，
Evaluate：
  accuracy : 0.3611111111111

In [7]:
# 评估结果
all_IDR_avg_disorder_evaluate = []

# 将21个区域的平均disorder值一起作为特征
all_R_H_IDR_disorder =[]
all_R_H_IDR_disorder.extend(disorder_R_H[0])
all_R_H_IDR_disorder.extend(disorder_R_H[1])

# 设置超参数  n_estimators：决策数的数量 默认100
param_dict = {"n_estimators": [100, 200, 400, 700], "max_depth": [5, 8, 15, 25, 30, 50, 100],"random_state":np.arange(0, 31, 5)}
for r in range(0,31):
    one_random_state = []
    # train_test_split 按比例划分原数据集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(all_R_H_IDR_disorder,input_lable,train_size=0.8,random_state = r)
    # 定义模型
    model = RandomForestClassifier(criterion="entropy")
    # 将模型加入超参数网格搜索和交叉验证     
    model = GridSearchCV(model, param_grid=param_dict, cv=5)
    # 训练数据集
    model = model.fit(Xtrain, Ytrain)
    # 最优的模型参数
    params = model.best_params_ 
    # 模型评估
    predict = model.predict(Xtest)
    cm = confusion_matrix(Ytest,predict)
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0
    
    one_random_state.append(accuracy)
    one_random_state.append(mcc)
    one_random_state.append(sens)
    one_random_state.append(spec)  
    all_IDR_avg_disorder_evaluate.append(one_random_state)
    print("数据集划分 random_state：" + str(r) + " 时，")
    print("Evaluate：")
    print("最优的模型参数是：" + str(params))
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")

数据集划分 random_state：0 时，
Evaluate：
最优的模型参数是：{'max_depth': 8, 'n_estimators': 400, 'random_state': 0}
  accuracy : 0.8055555555555556
  mcc : 0.6181073683361271
  sens : 73.33333333333333
  spec : 85.71428571428571
数据集划分 random_state：1 时，
Evaluate：
最优的模型参数是：{'max_depth': 8, 'n_estimators': 100, 'random_state': 0}
  accuracy : 0.7222222222222222
  mcc : 0.44272445820433437
  sens : 70.58823529411765
  spec : 73.68421052631578
数据集划分 random_state：2 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'n_estimators': 200, 'random_state': 10}
  accuracy : 0.8333333333333334
  mcc : 0.8528028654224418
  sens : 70.0
  spec : 100.0
数据集划分 random_state：3 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'n_estimators': 100, 'random_state': 5}
  accuracy : 0.7777777777777778
  mcc : 0.5142956348249516
  sens : 87.5
  spec : 70.0
数据集划分 random_state：4 时，
Evaluate：
最优的模型参数是：{'max_depth': 5, 'n_estimators': 100, 'random_state': 20}
  accuracy : 0.8333333333333334
  mcc : 0.5806324452973928
  sens : 81.81818181818183
  spec : 8